# Pandas Sports Team Exercise
This is a project I did when I taught myself pandas through Coursera's Intro to Data Science with Python course. The first 4 cells focus on the 4 big North American sports leagues. In each cell, I calculate the win loss per city. For instance, the win loss of NFL teams in LA will be different than just the win loss of teh Chargers as this also factors in the win loss of the Rams. The last cell can be used to compute the p-values of the correlations found.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# nhl_df=pd.read_csv("assets/nhl.csv")
# cities=pd.read_html("assets/wikipedia_data.html")[1]
# cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    #Retrieving and downloading data
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]

    #Setting columns/rows
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]


    #Cleaning cities names and dropping useless data
    cities['NHL'].replace('(\[[\w ]+\])', '', regex = True, inplace = True)
    cities['NHL'].replace('(\—)', np.nan, regex = True, inplace = True)
    cities['NHL'].replace('', np.nan, regex = True, inplace = True)
    cities['NHL'].dropna()

    #Sorting cities and fitlering out useless ones
    cities = cities.sort_values('Metropolitan area')
    cities.index = cities['Metropolitan area']
    cities = cities[['NHL', 'Population (2016 est.)[8]']].dropna()
    
    #Calculating win loss in NHL DF
    nhl_df['team'].replace("(\*)", '', regex = True, inplace = True)
    nhl_df.index = nhl_df['team']
    nhl_df = nhl_df[nhl_df['year'] == 2018]

    #Dropping team titles that are actually division titles
    nhl_df.drop('Atlantic Division', axis = 0, inplace = True)
    nhl_df.drop('Pacific Division', axis = 0, inplace = True)
    nhl_df.drop('Metropolitan Division', axis = 0, inplace = True)
    nhl_df.drop('Central Division', axis = 0, inplace = True)
    
    #Sorting index
    nhl_df.sort_index(ascending = True, inplace = True)

    #Setting cities to teams
    nhl_df['Metropolitan area'] = pd.Series(['Los Angeles', 'Phoenix', 'Boston', 'Buffalo', 'Calgary', 'Raleigh', 'Chicago',
    'Denver', 'Columbus', 'Dallas–Fort Worth', 'Detroit', 'Edmonton', 'Miami–Fort Lauderdale', 'Los Angeles', 'Minneapolis–Saint Paul',
    'Montreal', 'Nashville', 'New York City', 'New York City', 'New York City', 'Ottawa', 'Philadelphia', 'Pittsburgh', 'San Francisco Bay Area',
    'St. Louis', 'Tampa Bay Area', 'Toronto', 'Vancouver', 'Las Vegas', 'Washington, D.C.', 'Winnipeg'], index = nhl_df.index)

    #Calculating Win Loss
    nhl_df['W'] = nhl_df['W'].apply(lambda x: int(x))
    nhl_df['L'] = nhl_df['L'].apply(lambda x: int(x))
    nhl_df['winloss'] = nhl_df['W']/(nhl_df['W'] + nhl_df['L'])
    nhl_df1 = nhl_df.groupby('Metropolitan area').agg('mean')
    
    population_by_region = cities['Population (2016 est.)[8]'].dropna().astype(int) # pass in metropolitan area population from cities
    win_loss_by_region = nhl_df1['winloss'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    #Return correlation between pop and win loss by city
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

FileNotFoundError: [Errno 2] No such file or directory: 'assets/nhl.csv'

In [3]:
def nba_correlation():
    #Retrieiving and reading data
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    #Cleaning Cities:
    cities['NBA'].replace('(\[[\w ]+\])', '', regex = True, inplace = True)
    cities['NBA'].replace('(\—)', np.nan, regex = True, inplace = True)
    cities['NBA'].replace('', np.nan, regex = True, inplace = True)

    #Sorting and setting index
    cities = cities.sort_values('Metropolitan area')
    cities.index = cities['Metropolitan area']
    cities = cities.dropna()
    
    #Cleaning nba_df and sorting data
    nba_df['team'].replace('(\*\s\([0-9]*\)$)', '', regex = True, inplace = True)
    nba_df['team'].replace('(\([0-9]*\))','', regex = True, inplace = True)
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df.sort_values('team', inplace = True)
    ndb_df = nba_df.set_index('team')

    #Assigning cities to teams
    ct = ['Atlanta', 'Boston', 'New York City', 'Charlotte', 'Chicago', 'Cleveland', 'Dallas–Fort Worth', 
                               'Denver', 'Detroit', 'San Francisco Bay Area','Houston', 'Indianapolis', 'Los Angeles', 'Los Angeles',
                                'Memphis', 'Miami–Fort Lauderdale', 'Milwaukee', 'Minneapolis–Saint Paul', 'New Orleans', 
                                'New York City', 'Oklahoma City', 'Orlando', 'Philadelphia', 'Phoenix', 'Portland', 'Sacramento', 
                                'San Antonio', 'Toronto', 'Salt Lake City', 'Washington, D.C.']

    
    #Calculating win loss
    nba_df['City'] = pd.Series(ct, index = nba_df.index)
    nba_df['W/L%'] = nba_df['W/L%'].apply(lambda x: float(x))

    nba_df = nba_df.groupby('City').agg('mean')
    
    
    population_by_region = cities['Population (2016 est.)[8]'].astype(int) # pass in metropolitan area population from cities
    win_loss_by_region = nba_df['W/L%'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [4]:
def mlb_correlation(): 
    #Retreiving data and reading it
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    #Cleaning team names and corting
    cities['MLB'].replace('(\[[\w ]+\])', '', regex = True, inplace = True)
    cities['MLB'].replace('(\—)', np.nan, regex = True, inplace = True)
    cities['MLB'].replace('', np.nan, regex = True, inplace = True)
    cities = cities.sort_values('Metropolitan area')
    cities.index = cities['Metropolitan area']
    cities = cities.dropna()
    
    #Cleaing and seperating data for mlb_df:
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    mlb_df.sort_values('team', inplace = True)
    mlb_df = mlb_df.set_index('team')

    #Assigning cites to teams
    ct = ['Phoenix', 'Atlanta', 'Baltimore', 'Boston', 'Chicago', 'Chicago', 'Cincinatti', 'Cleveland', 'Denver', 'Detroit', 
         'Houston', 'Kansas City', 'Los Angeles', 'Los Angeles', 'Miami–Fort Lauderdale', 'Milwaukee', 'Minneapolis–Saint Paul',
         'New York City', 'New York City', 'San Francisco Bay Area', 'Philadephia', 'Pittsburgh', 'San Diego', 
         'San Francisco Bay Area', 'Seattle', 'St. Louis', 'Tampa Bay Area', 'Dallas–Fort Worth', 'Toronto', 'Washington, D.C.']

    #Calculating win loss
    mlb_df['City'] = pd.Series(ct, index = mlb_df.index)
    mlb_df['W-L%'] = mlb_df['W-L%'].apply(lambda x: float(x))
    mlb_df = mlb_df.groupby('City').agg('mean')
    
    population_by_region = cities['Population (2016 est.)[8]'].astype(int) # pass in metropolitan area population from cities
    win_loss_by_region = mlb_df['W-L%'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [5]:
def nfl_correlation(): 
    #Getting and reading data
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    #Cleaning city names
    cities['NFL'].replace('(\[[\w ]+\])', '', regex = True, inplace = True)
    cities['NFL'].replace('(\—)', np.nan, regex = True, inplace = True)
    cities['NFL'].replace('', np.nan, regex = True, inplace = True)
    cities = cities.sort_values('Metropolitan area')
    cities.index = cities['Metropolitan area']
    cities = cities.dropna()
    
    #Cleaning and seperating nfl_df data:
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df.sort_values('team', inplace = True)
    nfl_df['team'].replace('([*+]+)', '', regex = True, inplace = True)

    nfl_df = nfl_df.set_index('team')

    nfl_df = nfl_df[4:]

    #Setting cities to team
    ct = ['Phoenix', 'Atlanta', 'Baltimore', 'Buffalo', 'Charlotte', 'Chicago', 'Cincinnati', 'Cleveland', 'Dallas–Fort Worth',
         'Denver', 'Detroit', 'Green Bay', 'Houston', 'Indianapolis', 'Jacksonville', 'Kansas City', 'Los Angeles', 'Los Angeles', 
         'Miami–Fort Lauderdale', 'Minneapolis–Saint Paul', 'Boston', 'New Orleans', 'New York City', 'New York City', 
         'San Francisco Bay Area', 'Philadeplhia', 'Pittsburgh', 'San Francisco Bay Area', 'Seattle', 'Tampa Bay Area', 'Nashville', 
         'Washington, D.C.']

    #Cleaning and calculating win loss
    nfl_df = nfl_df.drop(['NFC East', 'NFC North', 'NFC West', 'NFC South'])
    nfl_df['City'] = pd.Series(ct, index = nfl_df.index)
    nfl_df['W-L%'] = nfl_df['W-L%'].apply(lambda x: float(x))
    nfl_df = nfl_df.groupby('City').agg('mean')
    
    population_by_region = cities['Population (2016 est.)[8]'].astype(int) # pass in metropolitan area population from cities
    win_loss_by_region = nfl_df['W-L%'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [ ]:
def sports_team_performance():
    '''
    Calculating and returns p values
    '''
    mlb_df=pd.read_csv("assets/mlb.csv")
    nhl_df=pd.read_csv("assets/nhl.csv")
    nba_df=pd.read_csv("assets/nba.csv")
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    return p_values